In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

import NamedTrajectories as NT

In [ ]:
traj = rand(NT.NamedTrajectory, 10);

- We can also define convert_arguments for a series, but series is in Makie, not MakieCore
- Not as easy to set Theme defaults this way

In [ ]:
function convert_arguments(
    P::Type{<:Makie.Series}, 
    traj::NT.NamedTrajectory,
    name::Symbol;
    transformation=nothing,
    transformation_name=nothing,
)
    if !isnothing(transformation)
        output_data = try
            stack(transformation.(eachcol(traj[name])))
        catch
            throw(ArgumentError("Transformation $(name)->$(transformation_name) failed."))
        end
    else
        output_data = traj[name]
    end

    return convert_arguments(P, NT.get_times(traj), output_data)
end

Makie.used_attributes(
    ::Type{<:Makie.Series}, 
    ::NT.NamedTrajectory, 
    ::Symbol
) = (:transformation, :transformation_name)

MakieCore.plottype(::NT.NamedTrajectory, ::Symbol) = Makie.Series

In [ ]:
f = Figure()
plot(f[1,1], traj, :x, transformation=x -> abs.(x), transformation_name=:abs)
f

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
plot!(ax, traj, :x, linewidth=3)
Legend(f[1, 2], ax)
plot(f[2, 1], traj, :u)
f